<a href="https://colab.research.google.com/github/IsharaSilva/Prediction-Recommedation-system-for-Potato-Cultivation/blob/main/Detection_of_Healthy_Unhealthy_Leaves.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
import numpy as np # dataAnalysis
import pandas as pd #multidimensional arrays
import matplotlib.pyplot as plt #data visualization
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from skimage import img_as_ubyte
from skimage.util import img_as_float


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
my_folder_path = 'https://drive.google.com/drive/folders/1J_XyVRFE15SzLjFtT4Z1OrabYlNeDHGQ?usp=share_link'

In [10]:
!git clone https://github.com/IsharaSilva/Prediction-Recommedation-system-for-Potato-Cultivation.git

fatal: destination path 'Prediction-Recommedation-system-for-Potato-Cultivation' already exists and is not an empty directory.


In [11]:
from google.colab import drive

In [12]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
import os
path = os.listdir('/content/drive/MyDrive/HealthyUnhealthy/Training')
classes = {'Healthy':0, 'Unhealthy':1}

FileNotFoundError: ignored

In [ ]:
x=[]
y=[]
for cls in classes:
    pth = 'F:/L4S1/FYP/Dataset/Training/'+cls
    for j in os.listdir(pth):
        img = cv2.imread(pth+'/'+j)
        img1 = cv2.resize(img, (500,500))#image resizing
        img1 = cv2.medianBlur(img1, 5)#apply median filter for remove noise
        img1 = img_as_ubyte(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)) #contrast improve
        #ret,img1 = cv2.threshold(img1,127,255,cv2.THRESH_TRUNC) #Threshould technique      
        x.append(img1)
        y.append(img)

plt.imshow(x[0]) #image after preprocessing